#Bag of Words Meets Bag of Popcorn#

This notebook will provide a tutorial in using Word2Vectors to get a deeper understanding of sentiment analysis. 

Some of the libraries we will use are:

- Beautiful Soup
- nltk
- logging

Imports

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)
import nltk.data

nltk.download()   

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [8]:
pip install --upgrade gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.0 MB ? eta -:--:--
   - -------------------------------------- 1.0/24.0 MB 3.6 MB/s eta 0:00:07
   --- ------------------------------------ 2.1/24.0 MB 4.2 MB/s eta 0:00:06
   ---- ----------------------------------- 2.6/24.0 MB 3.6 MB/s eta 0:00:06
   ------ --------------------------------- 3.9/24.0 MB 4.2 MB/s eta 0:00:05
   ------- -------------------------------- 4.7/24.0 MB 4.0 MB/s eta 0:00:05
   ---------- ----------------------------- 6.0/24.0 MB 4.4 MB/s eta 0:00:05
   ------------ --------------------------- 7.6/24.0 MB 4.8 MB/s eta 0:00:04
   --------------- ------------------------ 9.2/24.0 MB 5.1 MB/s eta 0:00:03
   ----------------- ---------------------- 10.5/24.0 MB 5.4 MB/s eta 0:00:03
   -------------------- ------------------- 12.3/24.0 MB 5.6 MB/s eta 0:00:03
   --------------------- ------------------ 12.8/24.0 MB 5.4 MB/s eta 0:00:03
   -------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


Load Data

In [ ]:
train = pd.read_csv("data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("data/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

print("Read {} labeled train reviews, {} labeled test reviews, and {} unlabeled reviews\n".format(
    train["review"].size, test["review"].size, unlabeled_train["review"].size))


In [ ]:
#punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_sentences( review, tokenizer, remove_stopwords=False ):
   
    #nltk tokenizer to split sentences
    raw_sentences = tokenizer.tokenize(review.strip())
   
    sentences = []
    for raw_sentence in raw_sentences:
        
        if len(raw_sentence) > 0:
            
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    
    return sentences

In [ ]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)
